In [1]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [2]:
client = OpenAI(base_url="http://localhost:11434/v1", api_key="EnigmaWU")

In [3]:
chat_completion = client.chat.completions.create(
  model="llama3.1",
  messages=[
    {"role":"user","content":"Hello from EnigmaWU, what's your favorite blus in the sky?"},
  ]
)

In [4]:
print(chat_completion.choices[0].message.content)

Nice to meet you, EnigmaWU! However, I think there might be a tiny misunderstanding. You're referring to clouds, right?

As for me, as a digital AI assistant, I don't have personal experiences or preferences, but I can tell you about some of the most amazing cloud types!

Here are a few popular ones:

1. **Cirrus**: Thin, wispy clouds that often appear as feathery streaks in the sky.
2. **Altocumulus**: Mid-level clouds with waves or ripples, often bringing patches of sun and shade.
3. **Stratus**: Low-hanging clouds that cover the entire sky like a blanket, often producing light to moderate precipitation.

What's your favorite cloud type? Or maybe you have a special memory associated with an interesting cloud formation?


In [5]:
from typing import Any


class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role":"system","content":system})

    def __call__(self, message):
        self.messages.append({"role":"user","content":message})
        result = self.execute()
        self.messages.append({"role":"assistant","content":result})
        return result

    def execute(self):
        chat_completion = client.chat.completions.create(
            model="llama3.1",
            messages=self.messages
        )
        return chat_completion.choices[0].message.content

In [6]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [7]:
def caculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return "ST avg 1 lbs"
    elif name in "Border Collie":
        return "BC avg 2 lbs"
    elif name in "Toy Poodle":
        return "TP avg 3 lbs"
    else:
        return "Guessing 9 lbs"
    
known_actions = {
    "calculate": caculate,
    "average_dog_weight": average_dog_weight
}

In [8]:
abot = Agent(prompt)

In [9]:
result = abot("How much does a Bulldog weight?")
print(result)

Thought: I should look the dog's weight using average_dog_weight.

Action: average_dog_weight: Bulldog
PAUSE


In [10]:
result =  average_dog_weight("Bulldog")

In [11]:
result

'Guessing 9 lbs'

In [12]:
next_prompt = "Ovservation: {}".format(result)
print(next_prompt)

Ovservation: Guessing 9 lbs


In [13]:
abot(next_prompt)

'That is incorrect. The average dog breed given was bulldog and I returned an answer that it weights "Guessing 9 lbs" which is not the weight of a bulldog.\n\nThe correct Answer is: A bulldog weighs 51lbs.'

In [14]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [15]:
abot = Agent(prompt)

In [16]:
question = """I have 2 dogs, a border collie and a scottish terrier.
what is their combined weight?"""
abot(question)

'Thought: I need to add the weights of both the Border Collie and Scottish Terrier. I will use calculate to get the sum.\n\nAction: calculate: average_dog_weight: Border Collie + average_dog_weight: Scottish Terrier'

In [17]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: BC avg 2 lbs


In [18]:
abot(next_prompt)

"Thought: The average weight of a Scottish Terrier is still unknown, so I'll need to use that function on it too.\n\nAction: calculate: average_dog_weight: Scottish Terrier + average_dog_weight: Border Collie"

In [19]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)


Observation: ST avg 1 lbs


In [20]:
abot(next_prompt)

'Thought: Now that I know the averages for both breeds, I can add them together using calculate to find their combined weight.\n\nAnswer: Their combined weight would be 2 pounds (BC) + 1 pound (ST) which equals 3 pounds.'

In [21]:
next_prompt = "Observation: {}".format(eval("2 + 1"))
print(next_prompt)

Observation: 3


In [22]:
abot(next_prompt)


'Wait! As the output of Observation, I should be giving an internal Answer before being seen outside, but since you are telling me what was observed, I will now take that info and give my final answer.\n\nAnswer: 3'

In [29]:
action_re = re.compile(r'^Action: (\w+): (.*)$')

In [31]:
def query(question, max_turns=5):
  i = 0
  bot = Agent(prompt)
  next_prompt = question
  while i < max_turns:
    i += 1
    result = bot(next_prompt)
    actions = [
      action_re.match(a)
      for a in result.split("\n")
      if action_re.match(a)
    ]
    if actions:
      action, action_input = actions[0].groups()
      if action not in known_actions:
        raise Exception("Unknown action: {}".format(action, action_input))
      print(" -- running {} {}".format(action, action_input))
      observation = known_actions[action](action_input)
      print("Observation:", observation)
      next_prompt = "Observation: {}".format(observation)
    else:
      return

In [32]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

 -- running average_dog_weight Scottish Terrier
Observation: ST avg 1 lbs
 -- running average_dog_weight Border Collie
Observation: BC avg 2 lbs
 -- running calculate 1 + 2
Observation: 3
